In [55]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class StrategyBacktest:
    def __init__(self, dataset_path, func=None, start_date=None):
        self.df = pd.read_csv(dataset_path, index_col=0, parse_dates=True)
        self.df = self.df.asfreq('D', method='ffill')
        self.func = func
        self.weights_history = []

        if start_date:
            self.df = self.df.loc[start_date:]

    def get_weight(self, df):
        if self.func:
            weight = np.array(self.func(df))
            if weight.sum() < 1:
                # Include cash as the last element to fill up to 1
                weight = np.append(weight, 1 - weight.sum())
            else:
                # Ensure the weights include cash as zero if the sum is exactly 1
                weight = np.append(weight, 0)
            return weight
        else:
            raise NotImplementedError("Weight function not provided")

    def calculate_returns(self, tickers=None, rebalance=False, n=1):
        if tickers:
            df = self.df[tickers]
        else:
            df = self.df

        pct_df = df.pct_change()
        account = 100
        self.weights_history = []
        weights = self.get_weight(df.iloc[:n])
        balance = [(np.array(weights[:-1]) * account).tolist()]  # 초기 잔고 계산에서 현금 제외
        cash = weights[-1] * account  # 초기 현금 잔고
        self.weights_history.append(weights.copy())

        portfolio_values = [sum(balance[0]) + cash]  # 초기 포트폴리오 가치에 현금 포함

        for i in range(n+1, len(df)):
            new_balance = np.array(balance[-1]) * (np.array(pct_df.iloc[i].values)+1)
            balance.append(new_balance.tolist())
            total_value = sum(new_balance) + cash  # 현금 포함 총 포트폴리오 가치 계산
            portfolio_values.append(total_value)

            # 가격 변동에 따른 자연스러운 비중 변화 반영
            weights = np.append(new_balance / total_value, cash / total_value)
            self.weights_history.append(weights)

            if rebalance and self.is_month_end(df.index[i]):
                weights = self.get_weight(df.iloc[:i,:])  # 리밸런싱을 통해 가중치 재계산
                new_balance = (np.array(weights[:-1]) * total_value).tolist()
                cash = weights[-1] * total_value
                balance[-1] = new_balance
                portfolio_values[-1] = sum(new_balance) + cash
                self.weights_history[-1] = weights.copy()

        tickers_with_cash = tickers if tickers else df.columns.tolist()
        tickers_with_cash.append('Cash')
        self.weights_df = pd.DataFrame(self.weights_history, index=df.index[n:], columns=tickers_with_cash)

        return pd.DataFrame(portfolio_values, index=df.index[n:], columns=['Portfolio'])

    def run(self, tickers=None, rebalance=False, func=None, benchmark='EWY', title=None, n=1):
        if func:
            self.func = func
        self.results = self.calculate_returns(tickers, rebalance, n)
        self.plot_results(benchmark, title)

        port = pd.DataFrame(self.calculate_performance_metrics(self.results), index=['My Portfolio'])
        df = self.df[benchmark]
        bench = pd.DataFrame(df.loc[self.results.index[0]:self.results.index[-1]])
        bench = pd.DataFrame(self.calculate_performance_metrics(bench), index=['Benchmark'])

        solution = pd.concat([port, bench], axis=0)
        return solution
        
    def is_month_end(self, date):
        return pd.to_datetime(date).is_month_end

    def get_benchmark(self, benchmark, result):
        df = self.df[benchmark]
        bench_returns = (df[result.index[0]:result.index[-1]].pct_change() + 1).cumprod()
        return bench_returns

    def plot_results(self, benchmark, title=None):
        fig = make_subplots(rows=3, cols=1, 
                            subplot_titles=("Portfolio Performance", "Drawdown", "Portfolio Weights"), 
                            vertical_spacing=0.1)

        bench_returns = self.get_benchmark(benchmark, self.results)
        bench_returns = pd.DataFrame(bench_returns, columns=[benchmark])

        cum_returns = (self.results.pct_change() + 1).cumprod()
        fig.add_trace(go.Scatter(x=cum_returns.index, y=cum_returns['Portfolio'], name='Cumulative Returns', line=dict(width=3)), row=1, col=1)
        fig.add_trace(go.Scatter(x=bench_returns.index, y=bench_returns[benchmark], name='Benchmark', line=dict(width=1)), row=1, col=1)

        rolling_max = cum_returns.cummax()
        bench_rolling_max = bench_returns.cummax()
        drawdowns = (cum_returns - rolling_max) / rolling_max
        bench_drawdowns = (bench_returns - bench_rolling_max) / bench_rolling_max
        fig.add_trace(go.Scatter(x=drawdowns.index, y=drawdowns['Portfolio'], name='My_Drawdown', line=dict(color='orange', width=3)), row=2, col=1)
        fig.add_trace(go.Scatter(x=bench_drawdowns.index, y=bench_drawdowns[benchmark], name='Bench_Drawdown', line=dict(color='red', width=1)), row=2, col=1)

        for i in range(self.weights_df.shape[1]):
            fig.add_trace(go.Scatter(x=self.weights_df.index, y=self.weights_df.iloc[:, i],
                                    stackgroup='one',
                                    name=self.weights_df.columns[i]), row=3, col=1)

        fig.update_yaxes(title_text="Cumulative Returns", row=1, col=1)
        fig.update_yaxes(title_text="Drawdown", row=2, col=1)
        fig.update_yaxes(title_text="Weights", row=3, col=1)

        if title!=None: name = title
        else: name = 'Strategy Performance'
        fig.update_layout(height=900, title_text=name, showlegend=True)
        fig.show()

    def calculate_performance_metrics(self, df):
        cumulative_returns = np.array(df / df.iloc[0] - 1)
        num_years = len(df) / 365
        cagr = (df.iloc[-1] / df.iloc[0]) ** (1 / num_years) - 1
        vol = df.pct_change().std() * np.sqrt(365)
        mdd = (df / df.cummax() - 1).min()
        sharpe_ratio = (cagr / vol)

        return {'cumulative_returns': cumulative_returns[-1] * 100, 'cagr': float(cagr) * 100, 'vol': float(vol) * 100, 'mdd': float(mdd) * 100, 'sharpe_ratio': float(sharpe_ratio)}


In [119]:
# 정적전략

def sixty_forty(df):
    # ['SPY', 'IEF']
    return [0.6, 0.4]

def coffeehouse(df):
    # ['SPY', 'IVE', 'IWM', 'IWN', 'EFA', 'VNQ', 'AGG']
    return [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.4]

def permanet(df):
    # ['SPY', 'TLT', 'GLD', 'BIL']
    return [0.25, 0.25, 0.25, 0.25]

def goldenbutterfly(df):
    # ['SPY', 'IWN', 'GLD', 'SHY', 'TLT']
    return [0.2, 0.2, 0.2, 0.2, 0.2]

def ivy(df):
    # ['SPY', 'EFA', 'AGG', 'DBC', 'VNQ']
    return [0.2, 0.2, 0.2, 0.2, 0.2]

def allseasons(df):
    # ['SPY', 'TLT', 'IEF', 'GLD', 'DBC']
    return [0.3, 0.4, 0.15, 0.075, 0.075]

In [128]:
sf = StrategyBacktest('dataset.csv', allseasons, start_date='2020-01-01')
sf.run(tickers=['SPY', 'TLT', 'IEF', 'GLD', 'DBC'], rebalance=False, title='All Seasons Portfolio', n=1)

,cumulative_returns,cagr,vol,mdd,sharpe_ratio
My Portfolio,12.814443,2.967400,10.513171,-22.479383,0.282255
Benchmark,9.798464,2.292927,28.953195,-49.734174,0.079194


In [116]:
# 동적전략

def globaltactical(df):
    # ['SPY', 'EFA', 'DBC', 'VNQ', 'IEF']
    moving = df.rolling(window=305).mean().iloc[-1]
    last = df.iloc[-1,:]

    weights = pd.Series(0, index=df.columns)

    weights[last>moving] = 0.2
    return weights.tolist()

def dualmomentum(df):
    # ['SPY', 'VEU', 'BIL', 'AGG']

    last = df.iloc[-1,:]
    past = df.iloc[-366,:]

    rtn = last/past-1
    
    if rtn['SPY'] > rtn['VEU'] and rtn['SPY'] > rtn['BIL']: return [1, 0, 0, 0]
    elif rtn['SPY'] < rtn['VEU'] and rtn['VEU'] > rtn['BIL']: return [0, 1, 0, 0]
    else: return [0, 0, 0, 1]

def flexible(df):
    # ['VTI', 'VEA', 'VWO', 'GSG', 'VNQ', 'BND', 'SHY']

    last = df.iloc[-1,:]
    past = df.iloc[-122,:]

    rtn = last/past-1
    r = rtn.rank(ascending=False).astype(int)

    check_r = rtn[rtn<0].index.tolist()

    returns = np.log(df / df.shift(1))
    daily_volatility = returns.std()

    v = daily_volatility.rank(ascending=True).astype(int)

    correlations = {}

    for asset in returns.columns:
        other_assets = returns.drop(columns=[asset])
        portfolio_returns = other_assets.mean(axis=1)
        
        correlation = returns[asset].corr(portfolio_returns)
        correlations[asset] = correlation

    c = pd.Series(correlations).rank(ascending=True).astype(int)

    score = r + 0.5*v + 0.5*c

    sorted_score = score.sort_values()
    weights = pd.Series(0, index=score.index)
    weights[sorted_score.index[:3]] = 1/3

    for k in check_r:
        if weights[k] > 0:
            weights[k] = 0
            weights[-1] += 1/3

    return weights.tolist()

def quintswitchingfiltered(df):
    # ['SPY', 'QQQ', 'EFA', 'EEM', 'TLT']

    last = df.iloc[-1,:]
    past = df.iloc[-92,:]

    rtn = last/past-1
    if rtn['SPY'] < 0 or rtn['QQQ'] < 0 or rtn['EFA'] < 0 or rtn['EEM'] < 0: return [0,0,0,0,1]
    
    sorted_rtn = rtn.sort_values(ascending=False)
    weights = pd.Series(0, index=rtn.index)
    weights[sorted_rtn.index[:3]] = 1/3
    return weights.tolist()

def vigilant(df):
    # ['SPY', 'QQQ', 'EFA', 'EEM', 'AGG', 'IEF', 'LQD']
    
    p0 = df.iloc[-1,:]
    p1 = df.iloc[-30,:]
    p3 = df.iloc[-92,:]
    p6 = df.iloc[-183,:]
    p12 = df.iloc[-366,:]

    filter = 12 * (p0 / p1) + 4 * (p0 / p3) + 2 * (p0 / p6) + (p0 / p12) - 19
    if filter['SPY'] < 0 or filter['QQQ'] < 0 or filter['EFA'] < 0 or filter['EEM'] < 0:
        sorted = filter[['SPY', 'QQQ', 'EFA', 'EEM']].sort_values(ascending=False)
        weights = pd.Series(0, index=filter.index)
        weights[sorted.index[:1]] = 1
        return weights.tolist()
    
    else:
        sorted = filter[['AGG', 'IEF', 'LQD']].sort_values(ascending=False)
        weights = pd.Series(0, index=filter.index)
        weights[sorted.index[:1]] = 1
        return weights.tolist()
    
def acceleratingdualmomentum(df):
    # ['SPY', 'SCZ', 'TLT', 'TIP']

    p0 = df.iloc[-1,:]
    p1 = df.iloc[-30,:]
    p3 = df.iloc[-92,:]
    p6 = df.iloc[-183,:]

    mom = p0/p1 + p0/p3 + p0/p6
    rtn = p0/p1-1

    if mom['SPY'] > mom['SCZ'] and mom['SPY'] > 0: return [1, 0, 0, 0]
    if mom['SPY'] < mom['SCZ'] and mom['SCZ'] > 0: return [0, 1, 0, 0]
    else:
        sorted = rtn[['TLT', 'TIP']].sort_values(ascending=False)
        weights = pd.Series(0, index=mom.index)
        weights[sorted.index[:1]] = 1
        return weights.tolist()

In [149]:
sf = StrategyBacktest('dataset.csv', acceleratingdualmomentum, start_date='2019-07-03')
sf.run(tickers=['SPY', 'SCZ', 'TLT', 'TIP'], rebalance=True, title='Accelerating Dual Momentum(ADM)', n=183)

,cumulative_returns,cagr,vol,mdd,sharpe_ratio
My Portfolio,59.899546,12.056753,22.346025,-33.717271,0.539548
Benchmark,9.798464,2.292927,28.953195,-49.734174,0.079194


In [13]:
a = pd.read_csv('dataset.csv', index_col=0, parse_dates=True)
a

,SPY,QQQ,IWM,VGK,EWJ,EWY,EWH,VEA,VWO,DBC,...,IEF,BND,SHY,BIL,IVE,IWN,EFA,VNQ,AGG,VEU
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,85.676888,40.198441,51.381485,29.699129,31.089293,38.221519,10.241995,22.460705,27.985737,22.663843,...,66.749863,52.926376,71.629654,81.801971,38.545849,43.991493,36.564690,25.830832,71.221840,28.801359
2010-01-04,87.129936,40.787140,52.648746,30.605782,31.887264,39.593456,10.503602,23.064909,28.941343,23.234581,...,66.915588,52.986988,71.724602,81.766289,39.163914,45.151138,37.523785,25.721134,71.304657,29.692940
2010-01-05,87.360565,40.787140,52.467712,30.556787,32.078781,39.609505,10.601710,23.078051,29.036905,23.262197,...,67.209389,53.141884,71.811028,81.748482,39.374805,44.984398,37.556854,25.692274,71.629074,29.686340
2010-01-06,87.422058,40.541130,52.418354,30.740566,32.206459,40.179134,10.634410,23.084621,29.214375,23.676443,...,66.938202,53.121716,71.819611,81.748482,39.469311,44.893444,37.715607,25.646080,71.587639,29.818424
2010-01-07,87.791100,40.567482,52.805107,30.544529,31.919184,39.360790,10.634410,23.038647,28.961826,23.381868,...,66.938202,53.081291,71.802345,81.748482,39.811081,45.424015,37.570076,25.923206,71.504814,29.640121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-08,498.320007,432.790009,196.149994,63.630001,66.389999,63.279999,15.960000,47.470001,40.470001,22.200001,...,94.660004,72.430000,81.839996,91.510002,175.630005,148.559998,75.040001,84.250000,97.660004,55.630001
2024-02-09,501.200012,437.049988,199.339996,63.849998,66.629997,63.669998,16.030001,47.639999,40.630001,22.260000,...,94.489998,72.370003,81.809998,91.519997,175.479996,150.330002,75.300003,84.379997,97.629997,55.880001
2024-02-12,500.980011,435.339996,202.960007,63.900002,66.970001,64.449997,16.200001,47.759998,40.810001,22.270000,...,94.580002,72.389999,81.820000,91.540001,176.619995,153.380005,75.430000,84.400002,97.650002,56.029999


In [31]:
b = a.iloc[-1,:]
c = a.iloc[-366,:]

In [32]:
d = b/c-1

In [105]:
d[b>c] = 0.2

In [106]:
d

SPY    0.200000
QQQ    0.200000
IWM    0.200000
VGK    0.200000
EWJ    0.200000
EWY    0.200000
EWH   -0.170464
VEA    0.200000
VWO    0.200000
DBC   -0.100056
GLD    0.200000
TLT   -0.128594
HYG    0.200000
LQD    0.200000
TIP   -0.025653
IEF   -0.030228
BND    0.200000
SHY    0.200000
BIL    0.200000
IVE    0.200000
IWN    0.200000
EFA    0.200000
VNQ   -0.041311
AGG    0.200000
VEU    0.200000
dtype: float64

In [103]:
d[d<0].index.tolist()

['EWH', 'DBC', 'TLT', 'TIP', 'IEF', 'VNQ']

In [83]:
d[['EWH', 'DBC', 'TLT']]

EWH   -0.170464
DBC   -0.100056
TLT   -0.128594
dtype: float64

In [40]:
# 일일 수익률 계산
returns = a.pct_change()

# 각 자산과 해당 자산을 제외한 나머지 자산의 포트폴리오 간 상관관계 저장
correlations = {}

# 각 자산에 대해 반복
for asset in returns.columns:
    # 해당 자산을 제외한 나머지 자산의 수익률
    other_assets = returns.drop(columns=[asset])
    # 동일 가중치로 나머지 자산의 평균 수익률 계산
    portfolio_returns = other_assets.mean(axis=1)
    
    # 해당 자산과 포트폴리오 간 상관관계 계산
    correlation = returns[asset].corr(portfolio_returns)
    correlations[asset] = correlation

In [43]:
pd.Series(correlations)

SPY    0.905274
QQQ    0.805823
IWM    0.860131
VGK    0.891131
EWJ    0.767645
EWY    0.796775
EWH    0.706642
VEA    0.937807
VWO    0.865961
DBC    0.466845
GLD    0.177971
TLT   -0.224531
HYG    0.802166
LQD    0.317090
TIP    0.097573
IEF   -0.132773
BND    0.157774
SHY    0.020999
BIL   -0.019489
IVE    0.887141
IWN    0.837523
EFA    0.929192
VNQ    0.759559
AGG    0.165504
VEU    0.949955
dtype: float64